In [ ]:
import requests
import pandas as pd
import tarfile
from sys import argv
from time import sleep
#from google.colab import files
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def queryMaker():
  data = pd.read_csv("./lista_palavroes.csv")
  arr = ""
  count = 0
  for a in data['badwords']:
      arr += a
      arr += ' OR '
      count += 1
      if len(arr) > 100:
        break
  print(count)
  return arr.strip(' OR')

print(queryMaker())

Definição da função **mine**

In [ ]:
#A função mine é responsavel por fazer uma requisição a API do Twitter
#e salvar os dados recebidos em um arquivo

#Defina a query da requisição na variável abaixo
#query= 'lang:pt place_country:br (#hashtag1 OR #hashtag2 OR (#hashtag3 #hashtag4))'
query="place_country:br ( {} )".format(queryMaker())

def mine(name, label, startDate, endDate, TOKEN, ):
        f = open(name+'.json','wt')
        #Aqui pode ser necessario mudar de 30day para fullarchive
        response = requests.get('https://api.twitter.com/1.1/tweets/search/fullarchive/'+label+'.json',
            params={'query':query,
                    'fromDate':startDate,
                    'toDate':endDate
                },
            headers={'Authorization':'bearer {}'.format(TOKEN)})
        f.write(response.text)
        return "request OK"

Coloque a sua **API_KEY** e **API_SECRET** no código abaixo para receber a acess_token, que sera usada para fazer as requisições

In [ ]:
API_KEY = ''   #'SUA KEY AQUI'
API_SECRET = ''   #'SUA SECRET KEY AQUI'

response = requests.post('https://api.twitter.com/oauth2/token',
                        auth=(API_KEY,API_SECRET),
                        params={'grant_type':'client_credentials'})
response.text

Definição da função **mine2**

In [ ]:
#A função mine2 é responsavel por fazer uma requisição a API do Twitter com a adicação do campo "next"
#e salvar os dados recebidos em um arquivo
def mine2(name, label, startDate, endDate, TOKEN,NEXT):
        f = open(name+'.json','wt')
        #Aqui pode ser necessario mudar de 30day para fullarchive
        response = requests.get('https://api.twitter.com/1.1/tweets/search/fullarchive/'+label+'.json',
            params={'query':query,
                    'fromDate':startDate,
                    'toDate':endDate,
                    'next':NEXT
                },
            headers={'Authorization':'bearer {}'.format(TOKEN)})
        f.write(response.text)
        return "request OK"

Setar os valores das variáveis e realizar a primeira requisição

In [ ]:
TOKEN = '' #access_token do bloco anterior
name = 'arq' #nome arquivo de saída da requisição
label = '' #nome do Dev environment
startDate = '201803010001' #yyyymmddhhmm
endDate = '201810302359' #yyyymmddhhmm

#Faz a primeira requisição para depois utilizar o next
ret = mine(name+'0', label, startDate, endDate, TOKEN)
print(ret)

In [ ]:
tam = 20 #valor final do range

#Cria o arquivo tar que vai conter todos os tweets
tt = tarfile.open("tuites.tar","w")
tt.add(name+'0'+".json")

requisicao_feita = open(name+'0'+".json", 'r') #Abre o arquivo da 1 requisição
string_json = requisicao_feita.read() #Lê o arquivo e salva numa variável
requisicao_feita.close() #Fecha arquivo
json_lido = json.loads(string_json) #Transforma o JSON lido em dicionário
NEXT = json_lido['next'] #Variável NEXT recebe o valor do campo 'next'

for aux2 in range(1,tam):
    ret = mine2(name + str(aux2), label, startDate, endDate, TOKEN, NEXT)
    tt.add(name+str(aux2)+".json")
    print(ret)
    requisicao_feita = open(name+str(aux2)+".json", 'r')
    string_json = requisicao_feita.read()
    json_lido = json.loads(string_json)
    NEXT = json_lido['next']
    sleep(1)

tt.close()

In [ ]:
tt.close()
aux2+=1
print('Contagem parou em: '+str(aux2))